In [1]:
# ==============================================================================
# 1. 라이브러리 설치 및 데이터 다운로드
# ==============================================================================

!pip install efficientnet_pytorch
!pip install torchmetrics
!pip install kagglehub  # KaggleHub 설치

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import kagglehub # 임포트
import time
import copy
from google.colab import drive
from collections import Counter # ⚠️ 클래스 카운트를 위한 라이브러리 추가
from google.colab import drive #

# 다운로드 및 경로 설정
path = kagglehub.dataset_download("shahadhossin567r7455/potato-leaf-disease-dataset")

print("다운로드된 데이터셋 경로:", path)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=f2e6a3132c2be6824c9bb147b65f607ad22fcd8969bb25a2bc1ab254ce40a4ba
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 32.1 MB/s eta 0:00:00
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


100%|██████████| 38.8M/38.8M [00:01<00:00, 30.0MB/s]

Extracting files...


다운로드된 데이터셋 경로: /root/.cache/kagglehub/datasets/shahadhossin567r7455/potato-leaf-disease-dataset/versions/1
사용 장치: cuda:0


In [20]:
# ==============================================================================
# 2. 데이터셋 경로 설정 (✨ 최종 확정: 실제 폴더 이름 반영)
# ==============================================================================
import os

# ⚠️ 'path' 변수는 다운로드 루트 경로를 담고 있다고 가정합니다.
DATA_ROOT_DOWNLOAD = path

# 🚨 [수정] 실제 폴더 이름인 'Potato Leaf DIsease'로 수정합니다.
DATA_ROOT_FOR_SPLIT = os.path.join(DATA_ROOT_DOWNLOAD, 'Potato Leaf DIsease')

print(f"✅ 최종 데이터 로드 경로 (DATA_ROOT_FOR_SPLIT): {DATA_ROOT_FOR_SPLIT}")


✅ 최종 데이터 로드 경로 (DATA_ROOT_FOR_SPLIT): /root/.cache/kagglehub/datasets/shahadhossin567r7455/potato-leaf-disease-dataset/versions/1/Potato Leaf DIsease


In [21]:
# ==============================================================================
# 3. 하이퍼파라미터 및 데이터 변환 설정 (모델 축소 적용)
# ==============================================================================
from efficientnet_pytorch import EfficientNet

# 하이퍼파라미터
BATCH_SIZE = 32 # B0 모델에 맞춰 배치 사이즈를 32로 늘려 속도 개선
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001 # 과적합 방지를 위해 학습률은 계속 낮게 유지
MODEL_NAME = 'efficientnet-b0' # ⭐️ 모델을 EfficientNet-B0로 축소

# EfficientNet-B0의 권장 입력 이미지 크기 (224x224로 자동 변경됨)
INPUT_SIZE = EfficientNet.get_image_size(MODEL_NAME)

# 데이터 증강 및 정규화
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.RandomResizedCrop(INPUT_SIZE),
        # 💡 [추가] 회전 및 색상 변화 추가
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 통계
    ]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [22]:
# ==============================================================================
# 4. 데이터 로드 및 DataLoader 생성 (수정: 단일 폴더 구조 → Random Split)
# ==============================================================================
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

print("\n데이터 로딩 및 안전한 90:10 분할 중...")

# 1. DATA_ROOT 전체를 하나의 데이터셋으로 로드합니다.
# ⚠️ [수정] DATA_ROOT 대신, 클래스 폴더가 들어있는 정확한 경로 변수를 사용해야 합니다.
#    이전 경로 설정 섹션에서 정의한 DATA_ROOT_FOR_SPLIT 변수를 사용합니다.
full_dataset = ImageFolder(DATA_ROOT_FOR_SPLIT, data_transforms['train']) # 💡 DATA_ROOT_FOR_SPLIT 사용
full_size = len(full_dataset)

# 2. 비율 설정 (90% 학습, 10% 검증)
train_size = int(0.9 * full_size)
val_size = full_size - train_size

# 3. 데이터셋 분리 (겹치지 않음을 보장)
train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)

# 4. DataLoader 재정의
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
    # 'val' 키에 검증 데이터셋(val_dataset)을 로드합니다.
    'val': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
}

# 5. 메타 정보 업데이트
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes
NUM_CLASSES = len(class_names)

print(f"클래스 개수: {NUM_CLASSES}")
print(f"클래스 이름: {class_names}")
print(f"✅ 새로운 학습 데이터 크기: {dataset_sizes['train']} (90%)")
print(f"✅ 새로운 검증 데이터 크기: {dataset_sizes['val']} (10%)")


데이터 로딩 및 안전한 90:10 분할 중...
클래스 개수: 7
클래스 이름: ['Early Blight', 'Fungal Diseases', 'Healthy', 'Late Blight', 'Plant Pests', 'Potato Cyst Nematode', 'Potato Virus']
✅ 새로운 학습 데이터 크기: 3150 (90%)
✅ 새로운 검증 데이터 크기: 350 (10%)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [23]:
# ==============================================================================
# 5. 모델 설정 및 가중치 부여 손실 함수 설정 (✨ B0 모델로 축소)
# ==============================================================================
from collections import Counter
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# ==============================================================================
# 1. 클래스 가중치 계산 (Class Imbalance 대처)
# ==============================================================================
from collections import Counter
# ⚠️ image_datasets['train'] 대신 full_dataset을 사용합니다.
# full_dataset은 '4. 데이터 로드' 섹션에서 정의되어야 합니다.

# full_dataset은 Subset이 아닌 ImageFolder 객체여야 target을 가져올 수 있습니다.
# 만약 full_dataset이 ImageFolder 객체였다면:
# train_targets = full_dataset.targets

# 🚨 full_dataset이 random_split 전의 ImageFolder 객체였고, 그 객체가 image_datasets['train']에 해당한다면
# 쌀 모델에서 'train' 데이터셋만 사용했으므로,
# '4. 데이터 로드' 섹션에서 정의된 `full_dataset` 객체를 사용해야 합니다.
# 현재 구조에서는 full_dataset이 ImageFolder 객체이므로 targets 속성이 있습니다.

train_targets = full_dataset.targets # 💡 [수정] full_dataset 객체를 사용해야 합니다.
class_counts = Counter(train_targets)
total_count = sum(class_counts.values())

class_weights = [total_count / class_counts[i] for i in range(NUM_CLASSES)]
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights / class_weights.min()

print(f"✅ 클래스 가중치 계산 완료: {class_weights.tolist()}")


# ----------------------------------------------------------------------
# 2. EfficientNet-B0 모델 로드 및 분류 레이어 명시적 교체 (수정)
# ----------------------------------------------------------------------

# ImageNet 가중치로 EfficientNet-B0 모델을 로드합니다.
# MODEL_NAME 변수는 이제 'efficientnet-b0'입니다.
model_ft = EfficientNet.from_pretrained(MODEL_NAME).to(device)

# 기존 분류 레이어(_fc)의 입력 피처 수 확인
num_ftrs = model_ft._fc.in_features

# 💡 [수정] 실제 데이터셋의 클래스 수에 맞춰 새로운 Sequential 레이어 정의 및 교체
#      (Dropout(0.5)을 추가하여 과적합 대처)
model_ft._fc = nn.Sequential(
    nn.Dropout(0.5), # 50%의 확률로 Dropout 적용
    nn.Linear(num_ftrs, NUM_CLASSES)
)

model_ft = model_ft.to(device) # 장치에 다시 올립니다.

print(f"✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 {num_ftrs} -> {NUM_CLASSES}로 성공적으로 교체되었습니다.")


# ----------------------------------------------------------------------
# 3. Loss, Optimizer, Scheduler 정의
# ----------------------------------------------------------------------

# 계산된 class_weights를 GPU/CPU 장치에 올리고 CrossEntropyLoss에 적용
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# AdamW 옵티마이저 사용 (수정된 LEARNING_RATE 적용)
optimizer_ft = optim.AdamW(model_ft.parameters(), lr=LEARNING_RATE)

# 학습률 스케줄러: 8 에포크마다 0.5배 감소로 완화
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.5)

print("✅ Loss, Optimizer, Scheduler 정의 완료.")


✅ 클래스 가중치 계산 완료: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Loaded pretrained weights for efficientnet-b0
✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 1280 -> 7로 성공적으로 교체되었습니다.
✅ Loss, Optimizer, Scheduler 정의 완료.


In [24]:
# ==============================================================================
# 6. 모델 학습 함수 (수정됨: 'test'를 'val'로 변경)
# ==============================================================================
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print(f'\nEpoch {epoch+1}/{num_epochs}'); print('-' * 10)

        # 💡 [수정] 'test' 대신 'val' 단계를 반복합니다.
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # 💡 [수정] 출력 메시지에서 'test'를 'val'로 변경
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 💡 [수정] 'val' 단계의 정확도가 최고 기록일 때 모델을 저장합니다.
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'-> New best model found! Saving weights with Acc: {best_acc:.4f}')

        epoch_time = time.time() - epoch_start_time
        print(f'Epoch Time: {epoch_time:.0f}초')

    time_elapsed = time.time() - since
    print(f'\n학습 완료! 총 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최고 검증 정확도: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [25]:
# ==============================================================================
# 7. 모델 학습 실행 및 저장
# ==============================================================================

# 모델 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

# Google Drive 마운트
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/disease_model.pth' # ⚠️ 파일 이름에 weighted 추가

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 최적 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")


Epoch 1/10
----------
train Loss: 1.6527 Acc: 0.4279
val Loss: 1.2831 Acc: 0.5829
-> New best model found! Saving weights with Acc: 0.5829
Epoch Time: 26초

Epoch 2/10
----------
train Loss: 0.9170 Acc: 0.7133
val Loss: 0.6323 Acc: 0.8229
-> New best model found! Saving weights with Acc: 0.8229
Epoch Time: 25초

Epoch 3/10
----------
train Loss: 0.6106 Acc: 0.7956
val Loss: 0.4189 Acc: 0.8743
-> New best model found! Saving weights with Acc: 0.8743
Epoch Time: 25초

Epoch 4/10
----------
train Loss: 0.5046 Acc: 0.8270
val Loss: 0.3530 Acc: 0.8886
-> New best model found! Saving weights with Acc: 0.8886
Epoch Time: 26초

Epoch 5/10
----------
train Loss: 0.4087 Acc: 0.8619
val Loss: 0.2651 Acc: 0.9029
-> New best model found! Saving weights with Acc: 0.9029
Epoch Time: 26초

Epoch 6/10
----------
train Loss: 0.3635 Acc: 0.8787
val Loss: 0.2451 Acc: 0.9257
-> New best model found! Saving weights with Acc: 0.9257
Epoch Time: 25초

Epoch 7/10
----------
train Loss: 0.3257 Acc: 0.8962
val Loss: 

MessageError: Error: credential propagation was unsuccessful

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# 구글 드라이브 마운트 (저장을 위해 필요합니다)
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/potato_disease_best_model.pth'

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ 모델 가중치가 /content/drive/MyDrive/potato_disease_best_model.pth에 저장되었습니다.
